### splice_pipeline.ipynb (fixed motif centring, parameterized GCF/Window for naming)

In [1]:
# 0. Imports & config

In [2]:
import random, re, bisect
from collections import defaultdict, Counter
from pathlib import Path

import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq

In [ ]:
# Optional. Helps for debugging:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

In [ ]:
# --- edit paths here ----------------------------------------------------------
ROOT = Path("data")
GCF = "GCF_000003195"

# Zea mays
# GCF = "GCF_902167145"
# FASTA = ROOT / "GCF_902167145.1_Zm-B73-REFERENCE-NAM-5.0_genomic.fna"
# GFF   = ROOT / "GCF_902167145.1_Zm-B73-REFERENCE-NAM-5.0_genomic.gff"

FASTA = ROOT / "GCF_000003195.3_Sorghum_bicolor_NCBIv3_genomic.fna"
GFF   = ROOT / "GCF_000003195.3_Sorghum_bicolor_NCBIv3_genomic.gff"

HALF = 200*(2**0)              # context size on each side of motif

OUT_PREP  = ROOT / f"data-{GCF}-half_window-{HALF}" / "preprocessed"
OUT_FINE  = ROOT / f"data-{GCF}-half_window-{HALF}"  / "fine_tuning"
OUT_PREP.mkdir(parents=True, exist_ok=True)
OUT_FINE.mkdir(parents=True, exist_ok=True)



In [67]:
OUT_PREP

WindowsPath('data/data-GCF_000003195-half_window-200/preprocessed')

##### 1. Basic helpers

In [68]:
DNA_COMPLEMENT = str.maketrans("ACGTNacgtn", "TGCANtgcan")
def rc(seq: str) -> str:
    return seq.translate(DNA_COMPLEMENT)[::-1]

def parse_fasta(fp: Path) -> dict:
    return {rec.id: str(rec.seq).upper() for rec in SeqIO.parse(str(fp), "fasta")}

def parse_gff_exons(fp: Path) -> pd.DataFrame:
    cols = ["seqid","source","type","start","end","score","strand","phase","attributes"]
    df   = pd.read_csv(fp, sep='\t', comment='#', header=None, names=cols)
    return df[df.type == "exon"].copy()

def tx_id(attr: str) -> str:
    for pat in (r'transcript_id=([^;]+)', r'Parent=transcript:([^;]+)'):
        m = re.search(pat, attr)
        if m: return m.group(1)
    return "unknown"

##### 2. Build splice‑site dataframe with **correct motif bounds**

In [69]:
def splice_sites_df(exons: pd.DataFrame, motif_len=2) -> pd.DataFrame:
    rows = []
    donor, acceptor = 0, 1
    for r in exons.itertuples():
        chrom, strand = r.seqid, r.strand
        tx = tx_id(r.attributes)
        exon_start = r.start # 1-based GFF start
        exon_end   = r.end   # 1-based GFF end

        # donor-exon -> intron boundary (first motif_len bases of intron)
        if strand == '+':
            # Intron starts after exon_end: [exon_end + 1, exon_end + motif_len]
            d_start = exon_end + 1
            d_end   = exon_end + motif_len
        else: # minus strand
            # Bio exon end is GFF exon_start. Intron starts before it.
            # Coords on reference strand: [exon_start - motif_len, exon_start - 1]
            d_start = exon_start - motif_len
            d_end   = exon_start - 1
        # Check for valid coords (must be > 0)
        if d_start > 0:
             rows.append(dict(id=f"{chrom}_{exon_end if strand == '+' else exon_start}_donor",
                              coord=exon_end if strand == '+' else exon_start, # The exon boundary coord
                              kind=donor, transcript=tx, strand=strand, chrom=chrom,
                              start=d_start, end=d_end)) # Motif coords (1-based)

        # acceptor-intron -> exon boundary (last motif_len bases of intron)
        if strand == '+':
            # Intron ends before exon_start: [exon_start - motif_len, exon_start - 1]
            a_start = exon_start - motif_len
            a_end   = exon_start - 1
        else: # minus strand
            # Bio exon start is GFF exon_end. Intron ends before it.
            # Coords on reference strand: [exon_end + 1, exon_end + motif_len]
            a_start = exon_end + 1
            a_end   = exon_end + motif_len
        # Check for valid coords (must be > 0)
        if a_start > 0:
            rows.append(dict(id=f"{chrom}_{exon_start if strand == '+' else exon_end}_acceptor",
                             coord=exon_start if strand == '+' else exon_end, # The exon boundary coord
                             kind=acceptor, transcript=tx, strand=strand, chrom=chrom,
                             start=a_start, end=a_end)) # Motif coords (1-based)
    return pd.DataFrame(rows)

##### 3. Window extraction (any motif_len)

In [70]:
def window_and_meta(df: pd.DataFrame, genome: dict, half=HALF) -> pd.DataFrame:
    def _grab(row):
        seq_chr = genome[row.chrom]
        ms0, me0 = row.start-1, row.end-1      # 0‑based inclusive motif bounds
        w0 = max(0, ms0 - half)
        w1 = min(len(seq_chr), me0 + half + 1) # exclusive
        snippet = seq_chr[w0:w1]
        if row.strand == '-':
            snippet = rc(snippet)
        exp = (me0 - ms0 + 1) + 2*half
        trunc = (w1 - w0) != exp
        return snippet, w0, w1, trunc

    vals = df.apply(_grab, axis=1)
    df['sequence']     = vals.str[0]
    df['win_start']    = vals.str[1]
    df['win_end']      = vals.str[2]
    df['is_truncated'] = vals.str[3]
    df['motif_len']    = df.end - df.start + 1
    # df['motif']        = df.apply(lambda r: r.sequence[half:half+r.motif_len], axis=1)

    # extract the true motif directly from genome, not from the (possibly truncated) snippet
    df['motif'] = df.apply( 
        lambda r: ( 
            genome[r.chrom][r.start-1 : r.end] 
            if r.strand == '+' 
            else rc(genome[r.chrom][r.start-1 : r.end]) 
        ), 
        axis=1 
    )
    return df

# def window_and_meta_old(df: pd.DataFrame, genome: dict, half=HALF) -> pd.DataFrame:
#     def _grab(row):
#         seq_chr = genome[row.chrom]
#         ms0, me0 = row.start-1, row.end-1      # 0‑based inclusive motif bounds
#         w0 = max(0, ms0 - half)
#         w1 = min(len(seq_chr), me0 + half + 1) # exclusive
#         snippet = seq_chr[w0:w1]
#         if row.strand == '-':
#             snippet = rc(snippet)
#         exp = (me0 - ms0 + 1) + 2*half
#         trunc = (w1 - w0) != exp
#         return snippet, w0, w1, trunc

#     vals = df.apply(_grab, axis=1)
#     df['sequence']     = vals.str[0]
#     df['win_start']    = vals.str[1]
#     df['win_end']      = vals.str[2]
#     df['is_truncated'] = vals.str[3]
#     df['motif_len']    = df.end - df.start + 1
#     df['motif']        = df.apply(lambda r: r.sequence[half:half+r.motif_len], axis=1)
#     return df

##### 4. Balanced (fast) negative sampler

In [ ]:
def negatives_sampler(df_pos: pd.DataFrame, genome: dict) -> pd.DataFrame: 
    """ 
    For each positive splice site in df_pos, sample (uniformly) exactly one different 
    occurrence of the same motif+strand as a negative.
    """ 
    # Build cumulative chromosome‐lengths so we can sample across the whole genome 
    from itertools import accumulate 
    chroms      = list(genome.keys()) 
    lengths     = [len(genome[ch]) for ch in chroms] 
    cum_lengths = list(accumulate(lengths)) 
    total_len   = cum_lengths[-1] 
    counts      = df_pos.groupby(['motif','strand']).size().to_dict()

    pos_set  = {(r.chrom, r.start-1, r.motif, r.strand) for r in df_pos.itertuples()} 
    neg_set  = set() 
    neg_rows = []
 
    import bisect 
    for (motif, strand), need in counts.items(): 
        target = motif if strand == '+' else rc(motif) 
        L      = len(target) 
        # uniformly sample [0 ... total_len-L] across concatenated genome 
        while need > 0: 
            r = random.randrange(0, total_len - L + 1) 
            # map 'r' back to (chrom, pos)
            idx = bisect.bisect_right(cum_lengths, r) 
            if idx == 0: 
                chrom = chroms[0] 
                pos   = r 
            else: 
                chrom = chroms[idx] 
                pos   = r - cum_lengths[idx-1] 
 
            key = (chrom, pos, motif, strand) 
            if key in pos_set or key in neg_set: 
                continue 
            seq = genome[chrom] 
            if seq[pos:pos+L] != target: 
                continue 
 
            neg_set.add(key) 
            s1 = pos + 1 
            neg_rows.append({ # Consider appending negative rows periodically instead
                'id':         f"{chrom}_{pos}_neg", 
                'coord':      pos, 
                'kind':       2, 
                'transcript': 'NEG', 
                'strand':     strand, 
                'chrom':      chrom, 
                'start':      s1, 
                'end':        s1 + L - 1, 
                'motif':      motif 
            }) 
            need -= 1
    return pd.DataFrame(neg_rows)

#### 6. Main

In [72]:
print("Loading genome & exons data")
genome = parse_fasta(FASTA)
exons  = parse_gff_exons(GFF)

Loading genome & exons data


In [51]:
print("Building splice-site dataframe")
df_pos = window_and_meta(splice_sites_df(exons), genome)

Building splice-site dataframe


In [52]:
df_donors = df_pos[df_pos['kind'] == 0].copy()

In [53]:
df_donors.head()

,id,coord,kind,transcript,strand,chrom,start,end,sequence,win_start,win_end,is_truncated,motif_len,motif
0,NC_012870.2_2713_donor,2713,0,XM_002463410.2,+,NC_012870.2,2714,2715,CTCGGGCCGCCAGAGGATGCATGTCAGCTGCGACCTGCTCGTCGGC...,2513,2915,False,2,TC
2,NC_012870.2_14503_donor,14503,0,XM_002466004.2,-,NC_012870.2,14501,14502,CATACTTGTTGAGAGGAAGAGGATCGGTCTGCTGCTACTAGCTGCG...,14300,14702,False,2,GT
4,NC_012870.2_14321_donor,14321,0,XM_002466004.2,-,NC_012870.2,14319,14320,ACCAAGTTCAAGATCAAGGTGAGTGGTAGCCCCCGATTGGTTCGAG...,14118,14520,False,2,GT
6,NC_012870.2_14017_donor,14017,0,XM_002466004.2,-,NC_012870.2,14015,14016,TGAAATGAAATTTTGAGTCCATCCTTGCTCACATCCGTCCCGCACC...,13814,14216,False,2,GT
8,NC_012870.2_13637_donor,13637,0,XM_002466004.2,-,NC_012870.2,13635,13636,TGCCCCGGTTATAACTGAGGATGGATTCCCTATCAAGGATCCATTG...,13434,13836,False,2,GT


In [54]:
df_donors.to_csv(OUT_PREP / f"df_donors-{GCF}-half_window-{HALF}.csv", index=False)

In [55]:
df_donors['motif'].value_counts(normalize=True)

motif
GT    0.828062
TT    0.028163
TG    0.022065
GC    0.019099
TA    0.018530
CT    0.014992
TC    0.014861
CA    0.012874
GA    0.010396
CC    0.007403
GG    0.005319
CG    0.005272
AT    0.004545
AA    0.003754
AG    0.002549
AC    0.002101
NN    0.000013
Name: proportion, dtype: float64

In [56]:
df_acceptors = df_pos[df_pos['kind'] == 1].copy()

In [57]:
df_acceptors.to_csv(OUT_PREP / f"df_acceptors-{GCF}-half_window-{HALF}.csv", index=False)

In [58]:
df_acceptors['motif'].value_counts(normalize=True)

motif
AG    0.838121
CC    0.018662
AA    0.014150
CA    0.013248
GC    0.012868
TC    0.012746
CT    0.011615
AC    0.011225
CG    0.010201
TT    0.010063
AT    0.008575
GA    0.008514
GG    0.008417
TG    0.007959
GT    0.007323
TA    0.006286
NN    0.000024
NC    0.000003
Name: proportion, dtype: float64

In [ ]:
print("generating balanced negatives")
df_neg = window_and_meta(negatives_sampler(df_pos, genome), genome) # window_and_meta(negatives(df_pos, genome))

generating balanced negatives


In [60]:
pd.concat([df_pos, df_neg]).to_csv(OUT_FINE / f"splice_sites_full_centered_balanced_correct-{GCF}-half_window-{HALF}.csv", index=False)
df_pos.to_csv(OUT_PREP / f"splice_sites_pos-{GCF}-half_window-{HALF}.csv", index=False)
df_neg.to_csv(OUT_PREP / f"splice_sites_neg-{GCF}-half_window-{HALF}.csv", index=False)

In [61]:
df_full = pd.concat([df_pos, df_neg])

In [62]:
df_full.head()

,id,coord,kind,transcript,strand,chrom,start,end,sequence,win_start,win_end,is_truncated,motif_len,motif
0,NC_012870.2_2713_donor,2713,0,XM_002463410.2,+,NC_012870.2,2714,2715,CTCGGGCCGCCAGAGGATGCATGTCAGCTGCGACCTGCTCGTCGGC...,2513,2915,False,2,TC
1,NC_012870.2_1683_acceptor,1683,1,XM_002463410.2,+,NC_012870.2,1681,1682,AAATCTTAGTTTTTTTTTCTAGAAATTTGGTTAAAGTTTAAAAAGT...,1480,1882,False,2,GC
2,NC_012870.2_14503_donor,14503,0,XM_002466004.2,-,NC_012870.2,14501,14502,CATACTTGTTGAGAGGAAGAGGATCGGTCTGCTGCTACTAGCTGCG...,14300,14702,False,2,GT
3,NC_012870.2_14887_acceptor,14887,1,XM_002466004.2,-,NC_012870.2,14888,14889,TCGTTATTTAATTTTACAGTATTGTTATGTTATTATGTGATCTGTA...,14687,15089,False,2,GC
4,NC_012870.2_14321_donor,14321,0,XM_002466004.2,-,NC_012870.2,14319,14320,ACCAAGTTCAAGATCAAGGTGAGTGGTAGCCCCCGATTGGTTCGAG...,14118,14520,False,2,GT


In [63]:
[len(x) for x in df_full.head(100)['sequence']]

[402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402,
 402]

##### === Motif-Strand Balance Summary ===

##### Classes balance across all motif+strand combinations.

| motif   | strand   |   pos_count |   neg_count |   diff | balanced   |
|:--------|:---------|------------:|------------:|-------:|:-----------|
|         | +        |           5 |           5 |      0 | True       |
|         | -        |          10 |          10 |      0 | True       |
| AA      | +        |        5661 |        5661 |      0 | True       |
| AA      | -        |        5623 |        5623 |      0 | True       |
| AC      | +        |        4732 |        4732 |      0 | True       |
| AC      | -        |        4620 |        4620 |      0 | True       |
| AG      | +        |      220406 |      220406 |      0 | True       |
| AG      | -        |      211328 |      211328 |      0 | True       |
| AT      | +        |        5026 |        5026 |      0 | True       |
| AT      | -        |        4757 |        4757 |      0 | True       |
| CA      | +        |        5930 |        5930 |      0 | True       |
| CA      | -        |        5830 |        5830 |      0 | True       |
| CC      | +        |        6636 |        6636 |      0 | True       |
| CC      | -        |        6929 |        6929 |      0 | True       |
| CG      | +        |        3953 |        3953 |      0 | True       |
| CG      | -        |        4029 |        4029 |      0 | True       |
| CT      | +        |        5880 |        5880 |      0 | True       |
| CT      | -        |        5940 |        5940 |      0 | True       |
| GA      | +        |        4308 |        4308 |      0 | True       |
| GA      | -        |        4103 |        4103 |      0 | True       |
| GC      | +        |        7927 |        7927 |      0 | True       |
| GC      | -        |        8270 |        8270 |      0 | True       |
| GG      | +        |        2827 |        2827 |      0 | True       |
| GG      | -        |        2882 |        2882 |      0 | True       |
| GT      | +        |      218701 |      218701 |      0 | True       |
| GT      | -        |      209083 |      209083 |      0 | True       |
| NN      | +        |           1 |           1 |      0 | True       |
| NN      | -        |           7 |           7 |      0 | True       |
| TA      | +        |        5782 |        5782 |      0 | True       |
| TA      | -        |        5671 |        5671 |      0 | True       |
| TC      | +        |        6329 |        6329 |      0 | True       |
| TC      | -        |        6237 |        6237 |      0 | True       |
| TG      | +        |        6311 |        6311 |      0 | True       |
| TG      | -        |        6002 |        6002 |      0 | True       |
| TT      | +        |        8323 |        8323 |      0 | True       |
| TT      | -        |        8164 |        8164 |      0 | True       |